In [7]:
import pandas as pd
import numpy as np

from typing import List
from scipy import stats

In [2]:
df_sales_detail = pd.read_csv('../data/2022-04-01T12_df_sales_detail.csv')
df_sales_detail.head()

,sale_id,good,price,date,user_id
0,1000001,mexican pizza,720,2022-02-04 10:00:24,1c1543
1,1000002,chefs pizza,840,2022-02-04 10:02:28,a9a6e8
2,1000002,orange juice,90,2022-02-04 10:02:28,a9a6e8
3,1000003,cheese pizza,600,2022-02-04 10:02:35,23420a
4,1000003,italian pizza,720,2022-02-04 10:02:35,23420a


In [3]:
df_sales = pd.read_csv('../data/2022-04-01T12_df_sales.csv')
df_sales.head()

,sale_id,date,count_pizza,count_drink,price,user_id
0,1000001,2022-02-04 10:00:24,1,0,720,1c1543
1,1000002,2022-02-04 10:02:28,1,1,930,a9a6e8
2,1000003,2022-02-04 10:02:35,3,1,1980,23420a
3,1000004,2022-02-04 10:03:06,1,1,750,3e8ed5
4,1000005,2022-02-04 10:03:23,1,1,870,cbc468


In [4]:
df_web_logs = pd.read_csv('../data/2022-04-01T12_df_web_logs.csv')
df_web_logs.head()

,user_id,page,date,load_time
0,f25239,m,2022-02-03 23:45:37,80.8
1,06d6df,m,2022-02-03 23:49:56,70.5
2,06d6df,m,2022-02-03 23:51:16,89.7
3,f25239,m,2022-02-03 23:51:43,74.4
4,697870,m,2022-02-03 23:53:12,66.8


In [46]:
df_web_logs1 = pd.read_csv('../data/2022-04-13T12_df_web_logs.csv')
df_web_logs1.head()

,user_id,page,date,load_time
0,f25239,m,2022-02-03 23:45:37,80.8
1,06d6df,m,2022-02-03 23:49:56,70.5
2,06d6df,m,2022-02-03 23:51:16,89.7
3,f25239,m,2022-02-03 23:51:43,74.4
4,697870,m,2022-02-03 23:53:12,66.8


In [5]:
df_exp_users = pd.read_csv('../data/2022-04-01_experiment_users.csv')
df_exp_users.head()

,user_id,pilot
0,0ffc65,0
1,b962b9,0
2,7ea63f,0
3,7f9a61,0
4,459e55,0


In [45]:
df_exp_users1 = pd.read_csv('../data/2022-04-13_experiment_users.csv')
df_exp_users1.head()

,user_id,pilot
0,c36b2e,0
1,20336e,0
2,034652,0
3,e98e3b,0
4,3f1105,0


## Топ-3 товаров по суммарной выручке

Определить топ-3 товара по суммарной выручке.

In [20]:
def get_top_item_revenue(data: pd.DataFrame, item_col: str, price_col: str, top_k: int) -> pd.DataFrame:
    data_revenue = data.groupby(item_col).agg(item_revenue=(price_col, 'sum')). \
            sort_values(by='item_revenue', ascending=False).reset_index()
    return data_revenue.iloc[:top_k]

In [21]:
get_top_item_revenue(df_sales_detail, item_col='good', price_col='price', top_k=3)

,good,item_revenue
0,chefs pizza,24558240
1,double pepperoni pizza,22558380
2,chicken bbq pizza,17622540


## Время от захода на сайт до покупки

Оценка среднего срока, который проходит от захода пользователя на сайт до совершения покупки. Будем считать, что заход на сайт относится к покупке, если он был совершён не ранее, чем за два часа до совершения покупки. Другими словами, для каждой покупки нужно посчитать время, которое проходит от покупки до первого захода на сайт того же пользователя в течение двух часов перед покупкой. И от полученных значений времени посчитать среднее.

In [25]:
df_sales.head()

,sale_id,date,count_pizza,count_drink,price,user_id
0,1000001,2022-02-04 10:00:24,1,0,720,1c1543
1,1000002,2022-02-04 10:02:28,1,1,930,a9a6e8
2,1000003,2022-02-04 10:02:35,3,1,1980,23420a
3,1000004,2022-02-04 10:03:06,1,1,750,3e8ed5
4,1000005,2022-02-04 10:03:23,1,1,870,cbc468


In [24]:
df_web_logs.head()

,user_id,page,date,load_time
0,f25239,m,2022-02-03 23:45:37,80.8
1,06d6df,m,2022-02-03 23:49:56,70.5
2,06d6df,m,2022-02-03 23:51:16,89.7
3,f25239,m,2022-02-03 23:51:43,74.4
4,697870,m,2022-02-03 23:53:12,66.8


In [69]:
def get_time_for_buy(data_sales: pd.DataFrame, data_web_logs: pd.DataFrame) -> int:
    user_sales = data_sales[['user_id', 'date']].rename(columns={'date': 'sale_date'})
    user_logs = data_web_logs[['user_id', 'date']].rename(columns={'date': 'event_date'})
    user_sales['sale_date'] = pd.to_datetime(user_sales['sale_date'])
    user_logs['event_date'] = pd.to_datetime(user_logs['event_date'])
    first_user_logs = user_logs.groupby('user_id')['event_date'].min().reset_index()

    user_sales_logs = user_sales.merge(right=first_user_logs, how='inner', on='user_id')
    user_sales_logs['diff_time'] = user_sales_logs['sale_date'] - user_sales_logs['event_date']
    user_sales_logs['diff_time_minute'] = user_sales_logs['diff_time'].dt.total_seconds() / 60
    user_buy_time = user_sales_logs[user_sales_logs['diff_time_minute'] <= 120]
    return round(user_buy_time['diff_time_minute'].mean())

In [70]:
get_time_for_buy(df_sales, df_web_logs)

17

## Удержание клиентов

Как много пользователей возвращается к нам из месяца в месяц. Какая доля пользователей, совершивших покупку в феврале, совершила покупку и в марте?

In [110]:
def retantion_next_month(data: pd.DataFrame, first_month: tuple[int, int], next_month: tuple[int, int]) -> float:
    users_data = data[['user_id', 'date']].copy()
    users_data['date'] = pd.to_datetime(users_data['date'])

    y1, m1 = first_month
    y2, m2 = next_month

    users_first_month = users_data[(users_data['date'].dt.year == y1) & (users_data['date'].dt.month == m1)]
    users_first_month = users_first_month[['user_id']].drop_duplicates()
    users_next_month = users_data[(users_data['date'].dt.year == y2) & (users_data['date'].dt.month == m2)]
    users_next_month = users_next_month[['user_id']].drop_duplicates()

    retantion = pd.merge(left=users_first_month, right=users_next_month, how='inner', on='user_id')
    return round(retantion['user_id'].count() / users_first_month['user_id'].count(), 2)

In [111]:
retantion_next_month(df_sales, first_month=(2022, 2), next_month=(2022, 3)) # 69662 

0.66

## Отличия до эксперимента
  
Возьмите те же группы, что и в эксперименте с изменением дизайна сайта, и проверьте значимость отличий средней выручки с пользователя на неделе перед экспериментом (c 2022.03.16 по 2022.03.23).

In [65]:
def exp_revenue(data_sales: pd.DataFrame,
                data_exp: pd.DataFrame,
                begin_date: str,
                end_date: str) -> float:
    
    df_sales = data_sales[['user_id', 'price', 'date']].copy()

    if begin_date:
        df_sales = df_sales[df_sales['date'] >= begin_date]

    if end_date:
        df_sales = df_sales[df_sales['date'] < end_date]

    user_revenue = df_sales.groupby('user_id')[['price']].sum().reset_index()
    user_revenue = user_revenue.merge(right=data_exp, how='right', on='user_id').fillna(0)

    control_group = user_revenue[user_revenue['pilot'] == 0]['price']
    test_group = user_revenue[user_revenue['pilot'] == 1]['price']

    return round(stats.ttest_ind(control_group, test_group).pvalue, 3)

In [66]:
pvalue = exp_revenue(df_sales, df_exp_users, begin_date='2022-03-16', end_date='2022-03-23')
print(pvalue)

0.199


## Среднее время между покупками


При выполнении прошлого задания вы могли заметить, что в данных много нулей. Это значит, что большая часть пользователей, совершивших покупку во время эксперимента, не совершала покупок на неделе до эксперимента. Интересно, как часто наши клиенты делают покупки?

Оцените среднее время между покупками. Возьмите всех клиентов, у которых 2 и более покупок. Вычислите время между покупками (для клиента с N покупками должно получиться N-1 значения времени). Объедините значения всех клиентов и вычислите среднее.

In [112]:
def mean_between_buy(sales_data: pd.DataFrame) -> float:
    user_time = df_sales[['user_id', 'date']].copy()
    user_time['date'] = pd.to_datetime(user_time['date'])
    user_time['count'] = user_time.groupby('user_id')[['date']].transform('count')
    user_time = user_time[user_time['count'] > 1]
    user_time = user_time.sort_values(['user_id','date']) \
        .groupby('user_id')['date'].diff().dropna()
    return round(user_time.dt.days.mean())

In [113]:
mean_between_buy(df_sales)

17

## Оценить необходимый размер групп
  
Допустим, мы хотим провести эксперимент, в который попадают клиенты, совершившие покупку во время эксперимента.

Метрика — средняя выручка с пользователя за время эксперимента;  
Продолжительность — одна неделя;  
Уровень значимости — 0.05;  
Допустимая вероятность ошибки II рода — 0.1;  
Ожидаемый эффект — 20 рублей.  
Оцените необходимый размер групп по данным о покупках за неделю с 21 по 28 февраля. Обратим внимание, что в выборку попадают события из полуинтервала [datetime(2022, 2, 21), datetime(2022, 2, 28)).

In [45]:
def get_sample_size(sales_data: pd.DataFrame,
                    epsilon: int,
                    begin_date: str=None,
                    end_date: str=None,
                    alpha: float=0.05,
                    beta: float=0.1) -> int:
    
    alp_inv = stats.norm.ppf(1 - alpha / 2)
    beta_inv = stats.norm.ppf(1 - beta)

    user_sales = sales_data[['user_id', 'date', 'price']].copy()
    if begin_date:
        user_sales = user_sales[user_sales['date'] >= begin_date]
    if end_date:
        user_sales = user_sales[user_sales['date'] < end_date]
    
    user_sales = user_sales.groupby('user_id')[['price']].sum()
    var_metric = user_sales['price'].var()
    
    sample_size = ((alp_inv + beta_inv) ** 2) * 2 * var_metric / (epsilon ** 2)
    return round(sample_size, -1)

In [46]:
get_sample_size(df_sales, 20, '2022-02-21', '2022-02-28')

34570.0

## MDE
  
В прошлом задании получилось, что необходимый размер групп больше имеющихся данных за одну неделю. Какой минимальный эффект мы можем отловить с теми же вероятностями ошибок на данных
с 21 по 28 февраля?

In [49]:
def get_mde(sales_data: pd.DataFrame,
            begin_date: str=None,
            end_date: str=None,
            alpha: float=0.05,
            beta: float=0.1) -> int:
    
    alp_inv = stats.norm.ppf(1 - alpha / 2)
    beta_inv = stats.norm.ppf(1 - beta)

    user_sales = sales_data[['user_id', 'date', 'price']].copy()
    if begin_date:
        user_sales = user_sales[user_sales['date'] >= begin_date]
    if end_date:
        user_sales = user_sales[user_sales['date'] < end_date]

    user_sales = user_sales.groupby('user_id')[['price']].sum()
    var_metric = user_sales['price'].var()
    sample_size = user_sales.shape[0] / 2

    epsilon = ((alp_inv + beta_inv) ** 2) * 2 * var_metric / sample_size
    return round(epsilon ** 0.5)

In [50]:
get_mde(df_sales, '2022-02-21', '2022-02-28')

33

## Численный эксперимент

Мы рассмотрели несколько вариантов добавления эффекта. Есть ли смысл думать о способе добавления эффекта при оценке вероятности ошибки II рода или все способы дают одинаковый результат? Результаты могут быть разными. Чтобы в этом убедиться, проведём численный эксперимент.

Допустим, в наш А/В-тест попадают все пользователи, совершавшие покупки до 28 марта.

 
Целевая метрика — средняя выручка с клиента за время эксперимента. Целевую метрику считаем на неделе с 21 по 28 марта. Уровень значимости — 0.05. Критерий — тест Стьюдента. Размер групп — 1000. Ожидаемый эффект — средняя выручка увеличится на 10%.

Нужно оценить вероятности ошибок II рода для трёх вариантов добавления эффекта:

1. Добавление константы ко всем значениям;

2. Умножение на константу всех значений;

3. Добавление константы к 2.5% значений.

In [20]:
def get_estimate_beta_error(data_sales: pd.DataFrame,
                   begin_date: str=None,
                   end_date: str=None):

    data_period = data_sales[['user_id', 'date', 'price']].copy()

    if begin_date:
        user_sales = data_period[data_period['date'] >= begin_date]
    else:
        user_sales = data_period.copy()
    
    if end_date:
        user_sales = user_sales[data_period['date'] < end_date]

        users_unique = data_period[data_period['date'] < end_date]
        users_unique = users_unique[['user_id']].drop_duplicates()
    
    mean_revenue_users = user_sales.groupby('user_id')[['price']].sum().reset_index()
    mean_revenue_users = mean_revenue_users.merge(right=users_unique,
                                                  how='right',
                                                  on='user_id').fillna(0)
    
    mean_price = mean_revenue_users['price'].mean()
    ratio1_beta_error = 0
    ratio2_beta_error = 0
    ratio3_beta_error = 0
    for _ in range(1000):
        # a_group = mean_revenue_users.sample(1000)['price']
        # b_group_one = mean_revenue_users.sample(1000)['price']
        a_group, b_group_one = np.random.choice(mean_revenue_users['price'], (2, 1000,), False)
        b_group_two = b_group_one.copy()
        b_group_three = b_group_one.copy()

        b_group_one += mean_price * 0.1
        b_group_two *= 1.1

        random_idx = np.random.choice(np.arange(1000), int(1000 * 0.025), False)
        b_group_three[random_idx] += mean_price * 0.1 * 1000 / len(random_idx)

        pvalue = stats.ttest_ind(a_group, b_group_one).pvalue
        ratio1_beta_error += int(pvalue > 0.05)
        pvalue = stats.ttest_ind(a_group, b_group_two).pvalue
        ratio2_beta_error += int(pvalue > 0.05)
        pvalue = stats.ttest_ind(a_group, b_group_three).pvalue
        ratio3_beta_error += int(pvalue > 0.05)
    
    ratio1_beta_error /= 1000
    ratio2_beta_error /= 1000
    ratio3_beta_error /= 1000
    print(f'Добавление константы: ошибка 2-го рода {ratio1_beta_error}')
    print(f'Умножение на константу: ошибка 2-го рода {ratio2_beta_error}')
    print(f'Добавление к 2.5% значений: ошибка 2-го рода {ratio3_beta_error}')


In [21]:
get_estimate_beta_error(df_sales,
                        begin_date='2022-03-21',
                        end_date='2022-03-28')

C:\Users\Vladislav\AppData\Local\Temp\ipykernel_6492\3193321133.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  user_sales = user_sales[data_period['date'] < end_date]


Добавление константы: ошибка 2-го рода 0.808
Умножение на константу: ошибка 2-го рода 0.822
Добавление к 2.5% значений: ошибка 2-го рода 0.818


## Оценка разных статистик
  
Проверка значимость отличий разных квантилей.

Данные эксперимента «Refactoring backend»: 2022-04-13/2022-04-13T12_df_web_logs.csv и 2022-04-13/experiment_users.csv. Эксперимент проводился с 2022-04-05 по 2022-04-12. Измерения времени обработки запросов считаем независимыми. При проверке используйте нормальный доверительный интервал.

In [59]:
def estimate_quantile(
        data_metric: pd.DataFrame,
        data_split: pd.DataFrame,
        begin_date: str,
        end_date: str,
        alpha: float=0.1,
        quantiles: list=[0.7, 0.74, 0.78,
                         0.82, 0.86, 0.9,
                         0.95, 0.99, 0.999, 0.9999]
    ):

    data_period = data_metric[['user_id', 'date', 'load_time']].copy()
    if begin_date:
        data_period = data_period[data_period['date'] >= begin_date]
    if end_date:
        data_period = data_period[data_period['date'] < end_date]
    
    data_period = data_period[['user_id', 'load_time']]
    data_period = data_period.merge(right=data_split, how='left', on='user_id')

    group_a = data_period[data_period['pilot'] == 0]['load_time'].values
    group_b = data_period[data_period['pilot'] == 1]['load_time'].values

    for q in quantiles:
        q_diff = np.quantile(group_b, q) - np.quantile(group_a, q)
        bootstrap_a = np.random.choice(group_a, (1000, len(group_a)), True)
        bootstrap_b = np.random.choice(group_a, (1000, len(group_a)), True)
        bootstrap_metric_a = np.quantile(bootstrap_a, q=q, axis=1)
        bootstrap_metric_b = np.quantile(bootstrap_b, q=q, axis=1)
        bootstrap_diff = bootstrap_metric_b - bootstrap_metric_a

        z_inv = stats.norm.ppf(1 - alpha / 2)
        bootstrap_std = np.std(bootstrap_diff)
        left = q_diff - z_inv * bootstrap_std
        right = q_diff + z_inv * bootstrap_std
        print(f"Estimation for q={q}: ({left}, {right})")

In [60]:
estimate_quantile(df_web_logs1,
                  df_exp_users1,
                  begin_date='2022-04-05',
                  end_date='2022-04-12')

Estimation for q=0.7: (0.7339525700707922, 1.2660474299292077)
Estimation for q=0.74: (0.30260036256109757, 0.8973996374388911)
Estimation for q=0.78: (-0.30058225872720273, 0.30058225872720273)
Estimation for q=0.82: (-1.0032219370047795, -0.3967780629951979)
Estimation for q=0.86: (-1.552518498219542, -0.8474815017804638)
Estimation for q=0.9: (-2.4814957976476864, -1.7185042023523025)
Estimation for q=0.95: (-3.8803996789033364, -3.029600321096518)
Estimation for q=0.99: (-7.345487137130331, -5.254512862869691)
Estimation for q=0.999: (-1552.7818567471772, 774.5114567470114)
Estimation for q=0.9999: (-338.253840871534, 401.6785608714012)
